In [10]:
#importing required libraries
import glob
from imageai.Detection import VideoObjectDetection
import natsort
import numpy as np
import os
import sys
import numpy as np
import cv2
import glob
from imageai.Detection import VideoObjectDetection
import os
import sys
from random import randint
from math import ceil, nan, sqrt
import natsort
import pandas as pd
import random
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV
import sys
import cv2
from random import randint
import re

In [5]:
def get_numbers_from_filename(filename):
    'utility function to get first number from a file name'
    return re.search(r'\d+', filename).group(0)

trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

def createTrackerByName(trackerType):
    
  # Create a tracker based on tracker name
    if trackerType == trackerTypes[0]:
        tracker = cv2.TrackerBoosting_create()
  
    elif trackerType == trackerTypes[1]: 
        tracker = cv2.TrackerMIL_create()
   
    elif trackerType == trackerTypes[2]:
        tracker = cv2.TrackerKCF_create()
  
    elif trackerType == trackerTypes[3]:
        tracker = cv2.TrackerTLD_create()
   
    elif trackerType == trackerTypes[4]:
        tracker = cv2.TrackerMedianFlow_create()
    
    elif trackerType == trackerTypes[5]:
        tracker = cv2.TrackerGOTURN_create()
            
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
        
    elif trackerType == trackerTypes[7]:
        tracker = cv2.TrackerCSRT_create()
        
    else:
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers are:')
        for t in trackerTypes:
            print(t)
     
    return tracker

In [3]:
def get_speeds_from_frame(video, boxes_list, frame_no, no_previous_frames, trackerType = "KCF"):

    '''
    Detects speed of each vehicle in the frame and returns and a list 
    containing these speeds

    Parameters:
    videoPath(string) : path of the video
    trackerType(string) : the openCV tracker type to be used  
    boxes_list(list) : list containing bounding boxes for all detected vehicles in the particular frame
    frame_no : frame no of the video for which you require directions

    '''
    frame_buffer = []
    #cv2.namedWindow(video, cv2.WINDOW_NORMAL)
    #cv2.resizeWindow(video, height=650, width=1156)
    cap = cv2.VideoCapture(video)

    for i in range(frame_no - no_previous_frames, frame_no + 1):

        cap.set(1,i)
        ret, frame = cap.read()
        frame_buffer.append(frame)

    multiTracker_back = cv2.MultiTracker_create()
    #multiTracker = cv2.MultiTracker_create()
    n = len(boxes_list)
    speeds = [0]*n
    x1 = [0]*n  #x coordinates of vehicles in frame_no
    y1 = [0]*n #y coordinates of vehicles in frame_no
    x2 = [0]*n  #x coordinates of vehicles in frame_no - no_previous_frames
    y2 = [0]*n #y coordinates of vehicles in frame_no - no_previous_frames

    frame = frame_buffer.pop()
    for i in range(n):
        box = boxes_list[i]
        x1[i] = (box[0]+2*box[2])/2
        y1[i] = (box[1]+2*box[3])/2
        multiTracker_back.add(createTrackerByName(trackerType), frame, box)
        #all the vehicles from boxes_list added in the traker instance
        #applying vehicle tracking in backward direction
    for j in range(no_previous_frames):
        frame = frame_buffer.pop()
        (success_back, boxes_back) = multiTracker_back.update(frame)

    for i in range(n):
        b = boxes_back[i]
        x2[i] = (b[0]+2*b[2])/2
        y2[i] = (b[1]+2*b[3])/2
        x_d = x2[i] - x1[i]
        y_d = y2[i] - y1[i]
        dist = (x_d**2 + y_d**2)**0.5
        speeds[i] = dist

    return np.array(speeds)


In [4]:
def get_speeds_from_video(video, detection_array, target_folder, no_previous_frames = 5):

    boxes_array = np.load(detection_array, allow_pickle=True)
    no_frames = boxes_array.shape[0]
    print("no_frames: ", no_frames)
    findex = get_numbers_from_filename(video)
    S = []
    for i in range(no_previous_frames):
        S.append([])

    for frame_no in range(5, no_frames):
        print("processing frame_no: ", frame_no)

        temp = boxes_array[frame_no]
        boxes_list = [(b[0],b[1],b[2]-b[0],b[3]-b[1]) for b in temp]

        speeds = get_speeds_from_frame(video, boxes_list, frame_no, no_previous_frames)
        S.append(speeds)
            
    S = np.array(S)
    print('saving speeds for video' + video)
    np.save(target_folder+'speeds'+str(findex),S)


In [11]:
vname = "C:/RoadCrossingAssistant/Data/Videos/video1.MOV"
fname = "C:/RoadCrossingAssistant/Data/Arrays_RetinaNet/array1.npy"

bounding_boxes = np.load(fname, allow_pickle=True) #loading the numpy array containing all detected vehicles
no_frames = bounding_boxes.shape[0]

frame_no = 35
temp = bounding_boxes[frame_no]
boxes_list = [(b[0],b[1],b[2]-b[0],b[3]-b[1]) for b in temp]

#print(get_speed_from_frame(vname, "KCF", boxes_list, frame_no))
get_speeds_from_video(vname, fname, 'C:/RoadCrossingAssistant/Data/Speeds_RetinaNet/')


In [4]:
vname = "C:/RoadCrossingAssistant/Data/Videos/video1.MOV"

cv2.namedWindow('video', cv2.WINDOW_NORMAL)
cv2.resizeWindow('video', height=650, width=1156)

cap = cv2.VideoCapture(vname) #video_name is the video being called
cap.set(1,25); # Where frame_no is the frame you want
ret, frame = cap.read() # Read the frame
cv2.imshow('video', frame) # show frame on window
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
def visualize_speeds_video(video, detection_array, speeds_array):

    cap = cv2.VideoCapture(video)
    cv2.namedWindow(video, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(video, height=650, width=1156)
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    detection_array = np.load(detection_array, allow_pickle=True)
    speeds_array = np.load(speeds_array,allow_pickle=True)

    frame_count = -1

    while(cap.isOpened()):

        success,frame = cap.read()
        #print(i)

        if success == True:

            if(frame_count == -1):
                while True:

                    key_init = cv2.waitKey(25)
                    cv2.imshow(video, frame)

                    if key_init == ord('a'):
                        break

            frame_count = frame_count + 1

            speeds_frame = speeds_array[frame_count]
            temp = detection_array[frame_count]
            boxes_frame = [(b[0],b[1],b[2]-b[0],b[3]-b[1]) for b in temp]
            no_boxes = len(boxes_frame)

            if(frame_count < 6):

                for i in range(no_boxes):
                    box = boxes_frame[i]
                    (x,y,w,h) = [int(v) for v in box]
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)

                cv2.imshow(video, frame)
                cv2.waitKey(100)        

            else:
                if(len(speeds_frame) != len(boxes_frame)):
                    raise Exception("mismatch in length of speeds_frame and boxes_frame")
                
                for i in range(no_boxes):
                    box = boxes_frame[i]
                    (x,y,w,h) = [int(v) for v in box]
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
                    cv2.putText(frame, str(speeds_frame[i]), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 3)
                cv2.imshow(video, frame)
                cv2.waitKey(100)            

        else:
            break

    
    cap.release()
    cv2.destroyAllWindows()

In [12]:
sname = 'C:/RoadCrossingAssistant/Data/Speeds_RetinaNet/speeds1.npy'
#visualize_speeds_video(vname, fname, sname)

In [25]:
def visualize_video(video, detection_array):

    cap = cv2.VideoCapture(video)
    cv2.namedWindow(video, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(video, height=650, width=1156)
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    detection_array = np.load(detection_array, allow_pickle=True)

    frame_count = -1

    while(cap.isOpened()):

        success,frame = cap.read()
        #print(i)

        if success == True:

            if(frame_count == -1):
                while True:

                    key_init = cv2.waitKey(25)
                    cv2.imshow(video, frame)

                    if key_init == ord('a'):
                        break

            frame_count = frame_count + 1

            temp = detection_array[frame_count]
            boxes_frame = [(b[0],b[1],b[2]-b[0],b[3]-b[1]) for b in temp]
            print("frame: ", frame_count)
            no_boxes = len(boxes_frame)
                
            
            for i in range(no_boxes):
                box = boxes_frame[i]
                (x,y,w,h) = [int(v) for v in box]
                cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
    
            cv2.imshow(video, frame)
            cv2.waitKey(100)            

        else:
            break

    
    cap.release()
    cv2.destroyAllWindows()

In [26]:
visualize_video(vname, fname)

frame:  0
frame:  1
frame:  2
frame:  3
frame:  4
frame:  5
frame:  6
frame:  7
frame:  8
frame:  9
frame:  10
frame:  11
frame:  12
frame:  13
frame:  14
frame:  15
frame:  16
frame:  17
frame:  18
frame:  19
frame:  20
frame:  21
frame:  22
frame:  23
frame:  24
frame:  25
frame:  26
frame:  27
frame:  28
frame:  29
frame:  30
frame:  31
frame:  32
frame:  33
frame:  34
frame:  35
frame:  36
frame:  37
frame:  38
frame:  39
frame:  40
frame:  41
frame:  42
frame:  43
frame:  44
frame:  45
frame:  46
frame:  47
frame:  48
frame:  49
frame:  50
frame:  51
frame:  52
frame:  53
frame:  54
frame:  55
frame:  56
frame:  57
frame:  58
frame:  59
frame:  60
frame:  61
frame:  62
frame:  63
frame:  64
frame:  65
frame:  66
frame:  67
frame:  68
frame:  69
frame:  70
frame:  71
frame:  72
frame:  73
frame:  74
frame:  75
frame:  76
frame:  77
frame:  78
frame:  79
frame:  80
frame:  81
frame:  82
frame:  83
frame:  84
frame:  85
frame:  86
frame:  87
frame:  88
frame:  89
frame:  90
frame:  9

In [17]:
detection_array = np.load(fname, allow_pickle=True)
print(detection_array.shape)
print(detection_array[:15])

(210,)
[list([[1408, 531, 1451, 614], [1284, 478, 1373, 575], [1340, 519, 1426, 640], [1525, 536, 1692, 693], [1121, 534, 1297, 719], [1519, 460, 1821, 713], [11, 263, 623, 607], [1677, 546, 1905, 1048]])
 list([[1415, 527, 1462, 588], [1275, 484, 1374, 577], [1326, 516, 1427, 643], [1099, 534, 1286, 723], [1520, 458, 1825, 720], [38, 263, 651, 607], [1678, 549, 1904, 1048]])
 list([[933, 472, 1025, 534], [1419, 494, 1470, 568], [1414, 527, 1464, 588], [1279, 479, 1386, 577], [1319, 527, 1418, 646], [1519, 536, 1687, 699], [1509, 457, 1897, 715], [1074, 542, 1283, 727], [60, 277, 688, 604], [1675, 547, 1904, 1047]])
 list([[934, 472, 1026, 534], [1422, 481, 1474, 538], [1414, 487, 1472, 574], [1290, 478, 1400, 584], [1292, 481, 1392, 587], [1358, 488, 1469, 592], [1413, 494, 1475, 589], [1300, 490, 1450, 626], [1307, 519, 1407, 647], [1053, 545, 1251, 733], [1512, 458, 1824, 726], [97, 278, 706, 611], [1673, 546, 1904, 1047]])
 list([[42, 352, 148, 514], [42, 353, 147, 515], [1401, 486